In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
from tqdm import tqdm
tqdm.pandas()
%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
from datasets import Dataset
import pandas as pd
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion,
  extract_first
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Google Cloud not set up, skipping import of providers.gemini_utils.generate_from_gemini_completion
Google Cloud not set up, skipping import of vertexai.preview.generative_models.Image and llms.generate_from_gemini_completion


/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/deepeval/__init__.py:41: UserWarning: You are using deepeval version 0.20.90, however version 0.21.23 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [4]:
from typing import Callable
from tqdm import tqdm

In [36]:
# subdependent 
from lm_act_eval.evaluation_harness.evaluators.sft.dataframe import DataFrameEvaluator
from lm_act_eval.evaluation_harness.helper_functions import multion
#


In [38]:
from lm_act_eval._init_conf import init_conf
hydra_cfg = init_conf('trajectory_eval-dev')
cfg = hydra_cfg.eval

Composed Configurations:
project: opentable
eval:
  sft:
    trajectory:
      data:
        path: lm_act_eval/.cache/five-star-trajectories/csv/data.csv
        columns:
          trajectory_id: session_id
          action_idx: idx_in_session
          'y': ground_truth
          y_': GPTV_generations
        extract_fs:
          QUERY:
            QUERY: null
          screenshot:
            screenshot: null
          GOAL:
            chat_completion_messages: parse_completion.parse_content
          HTML:
            DOM: opentable_extract_reservation_details
        logging:
          wandb:
            project: opentable
            result: lm_act_eval-run
          braintrust:
            project: multion_opentable
      metrics:
        opentable_html:
          evaluate_group_last: action_idx
          inputs:
          - HTML
          args:
          - null
        gpt-v:
          inputs:
          - GOAL
          - QUERY
          - screenshots
          args:
         

In [39]:
df = pd.read_csv(cfg.sft.trajectory.data.path)
entry = df.iloc[0]
print(
  df.chat_completion_messages.iloc[0][:100],
  df.columns
)
df.head(2)

[{'role': 'system', 'content': '**Goal**: Let\'s play a game - You are an expert agent named MULTI·O Index(['session_id', 'idx_in_session', '_id', 'event_id', 'parent_id',
       'event_type', 'event_name', 'config', 'children_ids', 'inputs',
       'outputs', 'error', 'start_time', 'end_time', 'metadata', 'feedback',
       'metrics', 'user_properties', 'source', 'duration', 'project_id',
       'tenant', 'QUERY', 'URL', 'DOM', 'RULES', 'USER_CONTEXT',
       'PREV_ACTIONS', 'chat_history', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'ground_truth', 'screenshot', 'prompt', 'internal_metadata'],
      dtype='object')


,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,PREV_ACTIONS,chat_history,CURRENT_TIME,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata
0,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,0,65f0c5d705cef8bdb70153a9,cc3d0d22-4759-43d8-a688-d174ca94f9dd,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'Find a restaurant with vegetarian o...,...,NaN,[],"Tuesday, March 12, 2024 at 4:14:11 PM CDT","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN: To find a restaurant with vegetarian opt...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN
1,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,1,65f0c60705cef8bdb70153ab,44b78fc2-24e2-40e5-b39d-e887a7129587,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'Find a restaurant with vegetarian o...,...,- I am searching for restaurants with vegetari...,"[{'role': 'user', 'content': 'Find a restauran...","Tuesday, March 12, 2024 at 4:15:02 PM CDT","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...","The search for ""vegetarian"" in St. Louis has y...",https://multion-client-screenshots.s3.us-east-...,NaN,NaN


In [40]:
cfg.sft.trajectory.metrics.opentable_html

{'evaluate_group_last': 'action_idx', 'inputs': ['HTML'], 'args': [None]}

In [45]:
from lm_act_eval.evaluation_harness.evaluators.metrics.opentable import opentable_reservation_html

In [24]:
from lm_act_eval.evaluation_harness.helper_functions import function_registry
function_registry.get('opentable_extract_reservation_details')(df.DOM.iloc[0])

{'restaurant_name': 'Unknown',
 'status': 'Unknown',
 'num_people': 0,
 'date_time': 'Unknown',
 'first_name': 'Unknown',
 'last_name': 'Unknown'}

In [50]:
from lm_act_eval.evaluation_harness.evaluators.sft.trajectory import TableTrajectoryEvaluator, AsyncTableTrajectoryEvaluator

df_evaluator = TableTrajectoryEvaluator(cfg.sft.trajectory)

Mapping fields to processing functions: 0it [00:00, ?it/s]/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/multion.py:159: UserWarning: Failed to parse json, returning content field
  
/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/multion.py:177: UserWarning: Unexpected format, returning suspicious content of type <class 'str'>, using as is
  ).strip()
Mapping fields to processing functions: 3it [00:02,  1.14it/s]


AttributeError: 'float' object has no attribute 'startswith'

In [ ]:
df_evaluator.df

,QUERY,screenshot,GOAL,HTML
session_id,,,,
ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,Find a restaurant with vegetarian options and ...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...,<coroutine object opentable_extract_reservatio...
ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,Find a restaurant with vegetarian options and ...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...,<coroutine object opentable_extract_reservatio...
ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,Find a restaurant with vegetarian options and ...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...,<coroutine object opentable_extract_reservatio...
ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,Find a restaurant with vegetarian options and ...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...,<coroutine object opentable_extract_reservatio...
61477dba-c090-4218-aaf4-6e20c4a66ba0,neste site clique em lançar e depois em despes...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...,<coroutine object opentable_extract_reservatio...
...,...,...,...,...
64cf461c-3d29-4ae1-b5d0-47af1814f9b8,open google search and look for joyfully read ...,https://multion-client-screenshots.s3.us-east-...,NA,<coroutine object opentable_extract_reservatio...
dc25be0f-4a81-44e8-9260-6ac22c5513b1,weather in sf,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...,<coroutine object opentable_extract_reservatio...
dc25be0f-4a81-44e8-9260-6ac22c5513b1,weather in sf,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...,<coroutine object opentable_extract_reservatio...


## Save/Load Data

In [ ]:
# traj_df_new = traj_df.merge(eligible_traj_df, how='left')
# traj_df_new.to_csv(dataset_path/'csv/data+gptv_eval.csv')

: 

In [ ]:
table_name = 'data+gptv_eval-eligible'
artifact_name = 'gptv_opentable-trajectory_eval'
eligible_traj_df.to_csv(dataset_path/f'csv/{table_name}.csv')

: 

In [ ]:
eligible_traj_df = pd.read_csv(
  dataset_path/f'csv/{table_name}.csv', index_col=0)
eligible_traj_df.index.name = None
eligible_traj_df.head(2)

: 

Logging

In [ ]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

: 

In [ ]:
opentable_artifact = wandb.Artifact("opentable_trajectories-gptv_eval", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(
  opentable_table_eligible, artifact_name)
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/f'csv/{table_name}.csv'))


: 

In [ ]:
# Start a W&B run to log data
run = wandb.init(project="trajectory_eval", reinit=True)
# Log the table to visualize with a run...
run.log({"opentable_gptv": opentable_table_eligible})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

: 